In [1]:
from datasets import load_dataset

dataset = load_dataset("RussianNLP/russian_super_glue", "danetqa")

dataset = dataset["validation"]
dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'passage', 'idx', 'label'],
    num_rows: 821
})

In [2]:
dataset[0]

{'question': 'Есть ли вода на марсе?',
 'passage': 'Гидросфера Марса — это совокупность водных запасов планеты Марс, представленная водным льдом в полярных шапках Марса, льдом над поверхностью, сезонными ручьями из жидкой воды и возможными резервуарами жидкой воды и водных растворов солей в верхних слоях литосферы Марса. Гидросфера Марса, вследствие господствующих низких температур на Марсе и нахождения запасов воды в твердом состоянии, также называется криолитосферой. Основная статья: История поисков воды на Марсе Марс весьма схож с Землей по многим показателям, что заставляло учёных XIX — начала XX века допускать, что на нём есть жизнь и есть жидкая вода. По мере роста объёма данных о планете, собранных различными методами, например, с помощью спектроскопических измерений, стало понятно, что воды в атмосфере Марса ничтожно малое количество, однако она всё же присутствует. Прежде всего внимание исследователей привлекли полярные шапки Марса, так как предполагалось, что они могут состоя

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

weights_path = "verbalist/model/models/verbalist_7b_v7/checkpoint-25900/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v7/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    trust_remote_code=True,
)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.48s/it]


In [2]:
inputs = ["Почему трава зеленая?"]

conversation = VerbalistConversation()
conversation.add_user_message(inputs[0])
prompt = conversation.get_prompt(tokenizer)
print("PROMPT", prompt)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=512,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.0,
    temperature=0.5,
    top_k=40,
    top_p=0.95,
    # do_sample=True,
)
output = generate(model, tokenizer, prompt, generation_config)
# print(inp)
print(output)

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Почему трава зеленая? </s> 
<s> bot


/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-pa

Зеленый цвет растений является результатом фотосинтеза, который происходит в листьях растений. В процессе фотосинтеза растения используют солнечный свет, воду и углекислый газ, чтобы произвести питательные вещества, такие как глюкозу и аминокислоты. В результате этого процесса растения вырабатывают кислород, который используется для дыхания, и выделяют кислород, который является основным продуктом фотосинтеза. Зеленый цвет растений является результатом наличия хлорофилла в листьях, который отвечает за получение зеленого цвета.


In [30]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import json
from pathlib import Path


class EvalRussianSuperGlue:
    def __init__(
        self,
        dataset_name="danetqa",
        model_type=None,
        model=None,
        base_folder=None,
        eval_name=None,
    ) -> None:
        self.dataset_name = dataset_name
        self.dataset = load_dataset("RussianNLP/russian_super_glue", dataset_name)
        self.dataset = self.dataset["validation"]
        self.model = model
        self.model_type = model_type

        self.base_folder = Path(base_folder)
        self.eval_name = eval_name

    def evaluate(self):
        functions_map = {
            "danetqa": self.danetqa,
        }
        functions_map[self.dataset_name]()

    def danetqa(self):
        print("danet")

        eval_folder = self.base_folder / f"{self.eval_name}"
        eval_folder.mkdir(exist_ok=True)
        output_file = eval_folder / f"DaNetQA.jsonl"
        if output_file.is_file():
            print("score file")
            predicts = []
            ground_true = []
            with open(output_file, "r") as f:
                lines = f.readlines()

                for i, line in enumerate(lines):
                    line = json.loads(line)
                    line["label"] = bool(
                        f'{line["label"][0].upper()}{line["label"][1:]}'
                    )
                    line["label"] = int(line["label"])
                    predicts.append(line["label"])
                    ground_true.append(dataset[i]["label"])
                    # print(line)
                    # break
                # print(predicts, ground_true)
                acc = accuracy_score(ground_true, predicts)
                print(f"Accuracy: {acc}")
        else:
            predicts = []
            ground_true = []
            with open(eval_folder / "DaNetQA.log", "w") as f:
                idxs = []
                for item in tqdm(self.dataset):
                    # print(item)
                    prompt = f"{item['question']}\nКонтекст: {item['passage']}\nИспользуя контекст, ответь на вопрос используя только да или нет."
                    result = self.get_answer(prompt=prompt)
                    print(prompt, file=f)
                    print(f"predict answer = {result}", file=f)
                    print(f"real answer = {item['label']}", file=f)

                    answer = None

                    if "да" in result:
                        answer = 1
                    elif "не" in result:
                        answer = 0
                    else:
                        answer = int(not bool(item["label"]))

                    predicts.append(answer)
                    idxs.append(item["idx"])
                    ground_true.append(item["label"])
                    # break
                acc = accuracy_score(ground_true, predicts)
                print(f"Accuracy: {acc}")

                with open(output_file, "w") as f:
                    for idx, predict in zip(idxs, predicts):
                        answer = {
                            "idx": idx,
                            "label": str(bool(predict)).lower(),
                        }
                        json.dump(answer, f)
                        f.write("\n")
                with open(eval_folder / "DaNetQA.txt", "w") as f:
                    f.write(acc)

    def get_answer(self, prompt):
        models_map = {
            "verbalist": self.verbalist_generation_1,
        }
        answer = models_map[self.model_type](prompt)
        answer = answer.strip()
        answer = answer.lower()
        return answer

    def verbalist_generation_1(self, prompt):
        conversation = VerbalistConversation()
        conversation.add_user_message(prompt)
        prompt = conversation.get_prompt(tokenizer)
        # print("PROMPT", prompt)
        generation_config = GenerationConfig(
            bos_token_id=1,
            eos_token_id=2,
            pad_token_id=0,
            max_new_tokens=512,
            # no_repeat_ngram_size=15,
            repetition_penalty=1.1,
            temperature=0.5,
            top_k=40,
            top_p=0.95,
            # do_sample=True,
        )
        output = generate(
            self.model,
            tokenizer,
            prompt,
            generation_config,
        )
        return output


evaluation = EvalRussianSuperGlue(
    dataset_name="danetqa",
    model_type="verbalist",
    model=model,
    base_folder="verbalist/evaluation/russian_super_glue/valid_evaluations/",
    eval_name="verbalist_7b_v7_checkpoint-25900",
)

evaluation.evaluate()

danet


FileNotFoundError: [Errno 2] No such file or directory: 'verbalist/evaluation/russian_super_glue/valid_evaluations/verbalist_7b_v7_checkpoint-25900/DaNetQA.log'